In [3]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("CustomerFeedbackAnalysis") \
    .getOrCreate()

# Define the path to your CSV file
csv_file_path = r"C:\Users\Abhay\Downloads\feedback_data.csv"

# Read the CSV file into a DataFrame
feedback_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
feedback_df.show()

# Print the schema of the DataFrame
feedback_df.printSchema()

# Stop the Spark session when done
spark.stop()


+-----------+----------------+------+--------------------+----------+
|Customer ID|Feedback Channel|Rating|             Comment|      Date|
+-----------+----------------+------+--------------------+----------+
|     C60065|           Email|     5|Not satisfied wit...|2024-11-15|
|     C30779|          Survey|     5|Fast and reliable...|2024-11-06|
|     C36228|          Survey|     3|The app is a bit ...|2024-11-10|
|     C11255|           Email|     1|Could improve pro...|2024-11-01|
|     C25224|           Email|     4|Fast and reliable...|2024-11-04|
|     C26897|           Email|     4|The app is a bit ...|2024-11-08|
|     C39081|           Email|     3|Not satisfied wit...|2024-11-03|
|     C68014|           Email|     1|Great experience ...|2024-11-15|
|     C65460|    Social Media|     4|Product quality c...|2024-11-12|
|     C66783|    Social Media|     5|Great experience ...|2024-11-05|
|     C42113|           Email|     4|I found the websi...|2024-11-06|
|     C11238|       

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col

# Create a Spark session
spark = SparkSession.builder \
    .appName("CustomerFeedbackAnalysis") \
    .getOrCreate()

# Define the path to your CSV file
csv_file_path = r"C:\Users\Abhay\Downloads\feedback_data.csv"

# Read the CSV file into a DataFrame
feedback_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
feedback_df.show()

# Cleaning comments
cleaned_feedback_df = feedback_df.withColumn(
    "Comment", 
    regexp_replace(col("Comment"), "[^a-zA-Z0-9\\s]", "")
).na.fill({"Comment": "No comment"})

# Show cleaned comments
cleaned_feedback_df.show()

# Stop the Spark session when done
spark.stop()


+-----------+----------------+------+--------------------+----------+
|Customer ID|Feedback Channel|Rating|             Comment|      Date|
+-----------+----------------+------+--------------------+----------+
|     C60065|           Email|     5|Not satisfied wit...|2024-11-15|
|     C30779|          Survey|     5|Fast and reliable...|2024-11-06|
|     C36228|          Survey|     3|The app is a bit ...|2024-11-10|
|     C11255|           Email|     1|Could improve pro...|2024-11-01|
|     C25224|           Email|     4|Fast and reliable...|2024-11-04|
|     C26897|           Email|     4|The app is a bit ...|2024-11-08|
|     C39081|           Email|     3|Not satisfied wit...|2024-11-03|
|     C68014|           Email|     1|Great experience ...|2024-11-15|
|     C65460|    Social Media|     4|Product quality c...|2024-11-12|
|     C66783|    Social Media|     5|Great experience ...|2024-11-05|
|     C42113|           Email|     4|I found the websi...|2024-11-06|
|     C11238|       

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col
from textblob import TextBlob

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CustomerFeedbackAnalysis") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

def analyze_sentiment(comment):
    """Analyze sentiment of the comment"""
    analysis = TextBlob(comment)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

# Register the UDF
sentiment_udf = udf(analyze_sentiment, StringType())

# Assume cleaned_feedback_df is already defined and is a valid Spark DataFrame
# Example DataFrame for demonstration
cleaned_feedback_df = spark.createDataFrame([
    (1, "Love the new updates!"),
    (2, "Too many bugs in the app...."),
    (3, "Fantastic I love it!"),
    (4, "Horrible service too slow!"),
], ["id", "Comment"])

# Apply the UDF
feedback_with_sentiment_df = cleaned_feedback_df.withColumn(
    "Sentiment", 
    sentiment_udf(col("Comment"))
)

feedback_with_sentiment_df.show()

# Stop Spark session when done
spark.stop()


Py4JJavaError: An error occurred while calling o203.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (LAPTOP-SDEV92GG executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

In [14]:
from kafka import KafkaProducer
import json
import random
import time
from datetime import datetime

# Kafka Producer
producer = KafkaProducer(bootstrap_servers='localhost:9092',
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

channels = ["Email", "Social Media", "Survey"]
ratings = [1, 2, 3, 4, 5]

while True:
    feedback = {
        "Customer ID": f"C{random.randint(10000, 99999)}",
        "Feedback Channel": random.choice(channels),
        "Rating": random.choice(ratings),
        "Comment": "The service was great!" if random.choice(ratings) > 3 else "Could improve product quality.",
        "Date": datetime.now().strftime("%Y-%m-%d")
    }
    producer.send('feedback_topic', feedback)
    print(f"Sent: {feedback}")
    time.sleep(1)  # Simulate a delay


SyntaxError: invalid syntax (simple.py, line 54)

In [12]:
pip install kafka


     ---------------------------------------- 0.0/207.2 kB ? eta -:--:--
     ----- ------------------------------- 30.7/207.2 kB 660.6 kB/s eta 0:00:01
     ---------- -------------------------- 61.4/207.2 kB 656.4 kB/s eta 0:00:01
     ---------------- -------------------- 92.2/207.2 kB 655.4 kB/s eta 0:00:01
     ---------------- -------------------- 92.2/207.2 kB 655.4 kB/s eta 0:00:01
     --------------------- -------------- 122.9/207.2 kB 602.4 kB/s eta 0:00:01
     --------------------------------- -- 194.6/207.2 kB 737.3 kB/s eta 0:00:01
     --------------------------------- -- 194.6/207.2 kB 737.3 kB/s eta 0:00:01
     --------------------------------- -- 194.6/207.2 kB 737.3 kB/s eta 0:00:01
     -----------------------------------  204.8/207.2 kB 518.8 kB/s eta 0:00:01
     ------------------------------------ 207.2/207.2 kB 466.7 kB/s eta 0:00:00
